![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [13]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as MSE

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV


# Import any additional modules and start coding below

SEED = 9

rental_df = pd.read_csv('rental_info.csv')
rental_df.info()

rental_df['rental_date'] = pd.to_datetime(rental_df['rental_date'])
rental_df['return_date'] = pd.to_datetime(rental_df['return_date'])

rental_df['rental_length'] = (rental_df['return_date'] - rental_df['rental_date']).dt.days

rental_df["deleted_scenes"] =  np.where(rental_df["special_features"].str.contains("Deleted Scenes"), 1,0)

rental_df['behind_the_scenes'] = np.where(rental_df["special_features"].str.contains("Behind the Scenes"), 1,0)

drop_cols = ["special_features", "rental_length", "rental_date", "return_date"]

X = rental_df.drop(drop_cols, axis=1)
y = rental_df["rental_length"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

lasso = Lasso(alpha=0.3, random_state=SEED)

lasso.fit(X_train, y_train)

lasso_coef = lasso.coef_

X_train_lasso, X_test_lasso, = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

ols = LinearRegression()
ols.fit(X_train_lasso, y_train)
y_test_pred = ols.predict(X_test_lasso)

mse_lr_lasso = MSE(y_test, y_test_pred)

params = {'n_estimators':np.arange(1, 101, 1), 'max_depth':np.arange(1, 11, 1)}

rf = RandomForestRegressor()

rs = RandomizedSearchCV(rf, param_distributions=params, cv=5, random_state=SEED)

rs.fit(X_train, y_train)

hyperparams = rs.best_params_

rf = RandomForestRegressor(n_estimators=hyperparams['n_estimators'], max_depth=hyperparams['max_depth'], random_state=SEED)

rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
mse_rf = MSE(y_test, rf_pred)

best_model = rf
best_mse = mse_rf

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB
